# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"><img src="../../images/icon102.png" width="38px"></img> **Hopsworks Feature Store** </span>

<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 01: Backfill Features to the Feature Store</span>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/logicalclocks/hopsworks-tutorials/blob/master/advanced_tutorials/air_quality/1_backfill_feature_groups.ipynb)


## 🗒️ This notebook is divided into the following sections:
1. Fetch historical data
2. Connect to the Hopsworks feature store
3. Create feature groups and insert them to the feature store

![tutorial-flow](../../images/01_featuregroups.png)

### <span style='color:#ff5f27'> 📝 Imports

In [ ]:
!pip install geopy folium streamlit-folium geopy --q

In [1]:
import datetime
import time
import requests
import json

import pandas as pd
import folium

from functions import *

import warnings
warnings.filterwarnings("ignore")

---

## <span style='color:#ff5f27'> 🌍 Representing the Target cities </span>

In [2]:
with open('target_cities.json') as json_file:
    target_cities = json.load(json_file)

## <span style='color:#ff5f27'> 🌫 Processing Air Quality data</span>

### [🇪🇺 EEA](https://discomap.eea.europa.eu/map/fme/AirQualityExport.htm)
#### EEA means European Environmental Agency

In [3]:
target_cities["EU"]

{'Amsterdam': [52.37, 4.89],
 'Athina': [37.98, 23.73],
 'Berlin': [52.52, 13.39],
 'Gdansk': [54.37, 18.61],
 'Kraków': [50.06, 19.94],
 'London': [51.51, -0.13],
 'Madrid': [40.42, -3.7],
 'Marseille': [43.3, 5.37],
 'Milano': [45.46, 9.19],
 'München': [48.14, 11.58],
 'Napoli': [40.84, 14.25],
 'Paris': [48.85, 2.35],
 'Sevilla': [37.39, -6.0],
 'Stockholm': [59.33, 18.07],
 'Tallinn': [59.44, 24.75],
 'Varna': [43.21, 27.92],
 'Wien': [48.21, 16.37]}

In [4]:
df_eu = pd.read_csv("data/backfill_pm2_5_eu.csv")

In [5]:
df_eu.isna().sum().sum()

0

In [6]:
print("Size of this dataframe:", df_eu.shape)

df_eu.sample(3)

Size of this dataframe: (63548, 3)


city_name        date  pm2_5
57287     Varna  2015-09-21   11.0
37368   München  2022-11-02    8.0
6233     Athina  2019-10-17   32.0

### [🇺🇸 USEPA](https://aqs.epa.gov/aqsweb/documents/data_api.html#daily)
#### USEPA means United States Environmental Protection Agency
[Manual downloading](https://www.epa.gov/outdoor-air-quality-data/download-daily-data)



In [7]:
target_cities["US"]

{'Albuquerque': [35.08, -106.65],
 'Atlanta': [33.75, -84.39],
 'Chicago': [41.88, -87.62],
 'Columbus': [39.96, -83.0],
 'Dallas': [32.78, -96.8],
 'Denver': [39.74, -104.98],
 'Houston': [29.76, -95.37],
 'Los Angeles': [34.05, -118.24],
 'New York': [40.71, -74.01],
 'Phoenix-Mesa': [33.66, -112.04],
 'Salt Lake City': [40.76, -111.89],
 'San Francisco': [37.78, -122.42],
 'Tampa': [27.95, -82.46]}

In [8]:
df_us = pd.read_csv("data/backfill_pm2_5_us.csv")

In [9]:
df_us.isna().sum().sum()

0

In [10]:
print("Size of this dataframe:", df_us.shape)

df_us.sample(3)

Size of this dataframe: (46037, 3)


date    city_name  pm2_5
959    2015-08-18  Albuquerque    5.8
9483   2018-12-31      Chicago    9.6
29726  2018-03-26     New York    6.5

### <span style="color:#ff5f27;">🏢 Processing special city - `Seattle`</span>
#### We need different stations across the Seattle. 
I downloaded daily `PM2.5` data manually [here](https://www.epa.gov/outdoor-air-quality-data/download-daily-data)

In [11]:
target_cities["Seattle"]

{'Bellevue-SE 12th St': [47.60086, -122.1484],
 'DARRINGTON - FIR ST (Darrington High School)': [48.2469, -121.6031],
 'KENT - JAMES & CENTRAL': [47.38611, -122.23028],
 'LAKE FOREST PARK TOWNE CENTER': [47.755, -122.2806],
 'MARYSVILLE - 7TH AVE (Marysville Junior High)': [48.05432, -122.17153],
 'NORTH BEND - NORTH BEND WAY': [47.49022, -121.77278],
 'SEATTLE - BEACON HILL': [47.56824, -122.30863],
 'SEATTLE - DUWAMISH': [47.55975, -122.33827],
 'SEATTLE - SOUTH PARK #2': [47.53091, -122.3208],
 'Seattle-10th & Weller': [47.59722, -122.31972],
 'TACOMA - ALEXANDER AVE': [47.2656, -122.3858],
 'TACOMA - L STREET': [47.1864, -122.4517],
 'Tacoma-S 36th St': [47.22634, -122.46256],
 'Tukwila Allentown': [47.49854, -122.27839],
 'Tulalip-Totem Beach Rd': [48.06534, -122.28519]}

In [12]:
df_seattle = pd.read_csv("data/backfill_pm2_5_seattle.csv")

In [13]:
df_seattle.isna().sum().sum()

0

In [14]:
print("Size of this dataframe:", df_seattle.shape)

df_seattle.sample(3)

Size of this dataframe: (46479, 3)


city_name        date  pm2_5
35059  LAKE FOREST PARK TOWNE CENTER  2021-11-02    7.3
1981          KENT - JAMES & CENTRAL  2013-07-04   11.4
12912         KENT - JAMES & CENTRAL  2016-04-14    3.0

In [15]:
df_seattle.city_name.value_counts()

NORTH BEND - NORTH BEND WAY                      3705
TACOMA - L STREET                                3696
SEATTLE - BEACON HILL                            3691
MARYSVILLE - 7TH AVE (Marysville Junior High)    3648
DARRINGTON - FIR ST (Darrington High School)     3614
SEATTLE - SOUTH PARK #2                          3577
TACOMA - ALEXANDER AVE                           3569
KENT - JAMES & CENTRAL                           3556
SEATTLE - DUWAMISH                               3439
Seattle-10th & Weller                            3097
LAKE FOREST PARK TOWNE CENTER                    2999
Tacoma-S 36th St                                 2574
Bellevue-SE 12th St                              2172
Tukwila Allentown                                2074
Tulalip-Totem Beach Rd                           1068
Name: city_name, dtype: int64

### <span style="color:#ff5f27;">🌟 All together</span>

In [16]:
df_air_quality = pd.concat([df_eu, df_us, df_seattle]).reset_index(drop=True)

In [17]:
df_air_quality.sample(5)

city_name        date  pm2_5
33604                                         Milano  2022-10-22   13.0
47414                                        Sevilla  2019-07-08   14.0
123811  DARRINGTON - FIR ST (Darrington High School)  2016-03-01    2.1
143516                        Tulalip-Totem Beach Rd  2020-08-23    1.4
30678                                         Milano  2014-10-18   32.0

In [18]:
df_air_quality.shape

(156064, 3)

In [19]:
df_air_quality.columns

Index(['city_name', 'date', 'pm2_5'], dtype='object')

---

## <span style='color:#ff5f27'> 🌦 Loading Weather Data from [Open Meteo](https://open-meteo.com/en/docs)

In [20]:
df_weather = pd.read_csv("data/backfill_weather.csv")

In [21]:
df_weather.city_name.value_counts()

Amsterdam                                        3767
Houston                                          3767
New York                                         3767
Phoenix-Mesa                                     3767
Salt Lake City                                   3767
San Francisco                                    3767
Tampa                                            3767
Bellevue-SE 12th St                              3767
DARRINGTON - FIR ST (Darrington High School)     3767
KENT - JAMES & CENTRAL                           3767
LAKE FOREST PARK TOWNE CENTER                    3767
MARYSVILLE - 7TH AVE (Marysville Junior High)    3767
NORTH BEND - NORTH BEND WAY                      3767
SEATTLE - BEACON HILL                            3767
SEATTLE - DUWAMISH                               3767
SEATTLE - SOUTH PARK #2                          3767
Seattle-10th & Weller                            3767
TACOMA - ALEXANDER AVE                           3767
TACOMA - L STREET           

In [22]:
df_weather.sample(3)

city_name        date  \
119477  DARRINGTON - FIR ST (Darrington High School)  2020-12-27   
142038                            SEATTLE - DUWAMISH  2020-12-28   
118711  DARRINGTON - FIR ST (Darrington High School)  2018-11-22   

        temperature_max  temperature_min  precipitation_sum  rain_sum  \
119477              4.5              1.9                4.7       4.5   
142038              7.2             -1.1                0.0       0.0   
118711             10.4              4.4               14.0      14.0   

        snowfall_sum  precipitation_hours  wind_speed_max  wind_gusts_max  \
119477          0.14                 16.0             8.8            40.3   
142038          0.00                  0.0             9.7            24.1   
118711          0.00                 17.0            10.8            58.0   

        wind_direction_dominant  
119477                      106  
142038                       52  
118711                      101

---

In [23]:
df_air_quality.date = pd.to_datetime(df_air_quality.date)
df_weather.date = pd.to_datetime(df_weather.date)

df_air_quality["unix_time"] = df_air_quality["date"].apply(convert_date_to_unix)
df_weather["unix_time"] = df_weather["date"].apply(convert_date_to_unix)

In [24]:
df_air_quality.date = df_air_quality.date.astype(str)
df_weather.date = df_weather.date.astype(str)

In [25]:
df_air_quality

city_name        date  pm2_5  \
0                                           Amsterdam  2013-01-01   14.0   
1                                           Amsterdam  2013-01-02    8.0   
2                                           Amsterdam  2013-01-03   12.0   
3                                           Amsterdam  2013-01-04   12.0   
4                                           Amsterdam  2013-01-05   14.0   
...                                               ...         ...    ...   
156059  MARYSVILLE - 7TH AVE (Marysville Junior High)  2023-03-30    7.9   
156060  MARYSVILLE - 7TH AVE (Marysville Junior High)  2023-03-31    3.7   
156061  MARYSVILLE - 7TH AVE (Marysville Junior High)  2023-04-01    3.4   
156062  MARYSVILLE - 7TH AVE (Marysville Junior High)  2023-04-02    3.1   
156063  MARYSVILLE - 7TH AVE (Marysville Junior High)  2023-04-03    4.4   

            unix_time  
0       1356998400000  
1       1357084800000  
2       1357171200000  
3       1357257600000  
4       1357344000000  
...               ...  
156059  1680130800000  
156060  1680217200000  
156061  1680303600000  
156062  1680390000000  
156063  1680476400000  

[156064 rows x 4 columns]

---

### <span style="color:#ff5f27;"> 🔮 Connecting to Hopsworks Feature Store </span>

In [26]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store() 

Copy your Api Key (first register/login): https://c.app.hopsworks.ai/account/api/generated
Connected. Call `.close()` to terminate connection gracefully.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/52403
Connected. Call `.close()` to terminate connection gracefully.


In [29]:
from great_expectations.core import ExpectationSuite, ExpectationConfiguration

expectation_suite = ExpectationSuite(expectation_suite_name="pmi_data")

expectation_suite.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_between",
        kwargs={
            "column":"pm2_5", 
            "min_value":"0.5",
            "max_value":"1000.0",
        }
    )
)
# feature

{"expectation_type": "expect_column_values_to_be_between", "meta": {}, "kwargs": {"column": "pm2_5", "min_value": "0.5", "max_value": "1000.0"}}

## <span style="color:#ff5f27;">🪄 Creating Feature Groups</span>

### <span style='color:#ff5f27'> 🌫 Air Quality Data

In [30]:
air_quality_fg = fs.get_or_create_feature_group(
    name="air_quality",
    description="Air Quality characteristics of each day",
    version=1,
    primary_key=["city_name"],  #'unix_time',
    online_enabled=False,
    expectation_suite=expectation_suite,
    event_time=["unix_time"],
)

In [31]:
air_quality_fg.insert(df_air_quality, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/52403/fs/51299/fg/54909
2023-06-02 09:36:22,943 INFO: 	1 expectation(s) included in expectation_suite.
Validation failed.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/52403/fs/51299/fg/54909


Uploading Dataframe: 0.00% |          | Rows 0/156064 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: air_quality_1_offline_fg_backfill
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/52403/jobs/named/air_quality_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7f5cd5c0f400>,
 {
   "success": false,
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expectations": 0,
     "unsuccessful_expectations": 1,
     "success_percent": 0.0
   },
   "meta": {
     "great_expectations_version": "0.14.12",
     "expectation_suite_name": "pmi_data",
     "run_id": {
       "run_time": "2023-06-02T08:36:22.943212+00:00",
       "run_name": null
     },
     "batch_kwargs": {
       "ge_batch_id": "8e1582f0-0120-11ee-8c38-00155d3208f1"
     },
     "batch_markers": {},
     "batch_parameters": {},
     "validation_time": "20230602T083622.943050Z",
     "expectation_suite_meta": {
       "great_expectations_version": "0.14.12"
     }
   },
   "results": [
     {
       "success": false,
       "expectation_config": {
         "expectation_type": "expect_column_values_to_be_between",
         "meta": {
           "expectationId": 47115
         },
         "kwargs": {
           "column": "pm2_5",
           "min_va

### <span style='color:#ff5f27'> 🌦 Weather Data

In [32]:
weather_fg = fs.get_or_create_feature_group(
    name='weather',
    description='Weather characteristics of each day',
    version=1,
    primary_key=['city_name'], #'unix_time'
    online_enabled=False,
    event_time=["unix_time"]
) 

In [33]:
weather_fg.insert(df_weather, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/52403/fs/51299/fg/54910


Uploading Dataframe: 0.00% |          | Rows 0/169515 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: weather_1_offline_fg_backfill
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/52403/jobs/named/weather_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7f5cd5c2bc10>, None)